# Optuna & Azure Machine Learning service 

PFN のハイパーパラメータチューニングのライブラリ [Optuna](https://preferred.jp/en/news/tag/optuna/) を Azure Machine Learning service で利用するサンプルノートブックです。

## Azure ML service Python SDK インポート

Azure Machine Learning の専用 Python SDK をインポートしておきます。

In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment

In [2]:
# バージョンの確認
print(azureml.core.VERSION)

1.0.65


## Azure Machine Learning Workspace への接続

Azure Machine Learning service との接続を行います。Azure に対する認証が必要です。

In [3]:
ws = Workspace.from_config()
print(ws.resource_group, ws.name)

mlservice azureml


## Experiment の設定

実験名の指定。実験管理の仕組みで利用します。

In [4]:
from azureml.core import Experiment
experiment_name = 'optuna-keras-mnist'
experiment = Experiment(workspace = ws, name = experiment_name)

## Optuna 用 Database への接続情報

結果の再利用や並列化のための Azure Database for MySQL の接続情報を取得します。

In [5]:
# 環境変数などから取得することを推奨
user = ""
dbname = ""
host = ""
dbname = ""
port = "3306"

In [6]:
# See details about Secret in https://docs.microsoft.com/ja-JP/azure/machine-learning/service/how-to-use-secrets-in-runs
keyvault = ws.get_default_keyvault()

In [7]:
# # Secrets の設定
# keyvault.set_secret(name="user", value = user)
# keyvault.set_secret(name="password", value = password )
# keyvault.set_secret(name="host", value = host)
# keyvault.set_secret(name="dbname", value = dbname)

In [8]:
# Secrets から値の取得
user = keyvault.get_secret(name="user")
password = keyvault.get_secret(name="password")
host = keyvault.get_secret(name="host")
dbname = keyvault.get_secret(name="dbname")

## study の作成

In [9]:
import optuna
import mysql.connector
from mysql.connector import errorcode

optuna.create_study(storage="mysql+mysqlconnector://{0}:{1}@{2}/{3}".format(user, password, host, dbname),
                    pruner=False,
                    study_name="keras-mnist",
                    direction='maximize',
                    load_if_exists=True,
                   )

[I 2019-10-20 00:12:25,292] A new study created with name: keras-mnist


## 学習スクリプト

In [10]:
%%writefile keras_mnist.py

from __future__ import print_function
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation
from keras.optimizers import SGD
from keras.utils import np_utils
import tensorflow as tf
from tensorflow.python.keras.utils.vis_utils import plot_model

import optuna

from azureml.core import Run

NB_EPOCH = 20
BATCH_SIZE = 32
NB_CLASSES = 10 #number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.3 # Training and Validation data ratio
DROPOUT = 0.3

class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('accuracy')))
        self.run.log(name="validation_acc", value=float(logs.get('val_accuracy')))
        

def objective(trial):
    
    lr = trial.suggest_loguniform('c', 1e-5, 1e-3)
    
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    RESHAPED = 784
    
    X_train = X_train.reshape(60000,RESHAPED)
    X_test = X_test.reshape(10000,RESHAPED)
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
        
    X_train /= 255
    X_test /= 255 
    
    Y_train = np_utils.to_categorical(y_train, NB_CLASSES)
    Y_test = np_utils.to_categorical(y_test, NB_CLASSES)
    
    model = Sequential()
    model.add(Dense(N_HIDDEN, input_shape = (RESHAPED,)))
    model.add(Activation('relu'))
    model.add(Dropout(DROPOUT))
    model.add(Dense(N_HIDDEN))
    model.add(Activation('relu'))
    model.add(Dropout(DROPOUT))
    model.add(Dense(NB_CLASSES))
    model.add(Activation('softmax'))
    model.summary()
    
    callbacks = list()

    OPTIMIZER = SGD(lr=lr) #SGD Optimizer
    model.compile(loss ='categorical_crossentropy', optimizer = OPTIMIZER, metrics=['accuracy'])


    callbacks.append(RunCallback(run))
    
    model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE, 
          epochs=NB_EPOCH, 
          callbacks= callbacks,
          verbose = 1, 
          validation_split=VALIDATION_SPLIT)
    

    score = model.evaluate(X_test, Y_test, verbose= 1)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

    # log a single value
    run.log("Final test loss", score[0])
    run.log('Final test accuracy', score[1])



    # create a ./outputs/model folder in the compute target
    # files saved in the "./outputs" folder are automatically uploaded into run history
    os.makedirs('./outputs/model', exist_ok=True)

    # serialize NN architecture to JSON
    model_json = model.to_json()
    
    # save model JSON
    with open('./outputs/model/model.json', 'w') as f:
        f.write(model_json)
        
    # save model weights
    model.save_weights('./outputs/model/model.h5')
    print("model saved in ./outputs/model folder")
    
    return score[1]



if __name__ == '__main__':
    
    # start an Azure ML run
    run = Run.get_context()
    
    print("Keras version:", keras.__version__)
    print("Tensorflow version:", tf.__version__)
    
    user = run.get_secret(name="user")
    password = run.get_secret(name="password")
    host = run.get_secret(name="host")
    dbname = run.get_secret(name="dbname")
    
    study = optuna.load_study(study_name='keras-mnist', storage="mysql+mysqlconnector://{0}:{1}@{2}/{3}".format(user, password, host, dbname))
    study.optimize(objective, n_trials=5)

    print(study.best_trial)

Writing keras_mnist.py


## 学習環境の準備

Machine Learning Compute は予め作成しておきます。

In [11]:
# 予め optuna-cluster という名称の Machine Learning Compute を作成しておく
from azureml.core.compute import ComputeTarget, AmlCompute
cpu_cluster = ComputeTarget(workspace=ws, name="optuna-cluster")

In [12]:
# ip address などの情報
cpu_cluster.list_nodes()

[]

In [13]:
# See https://docs.microsoft.com/ja-JP/azure/machine-learning/service/how-to-use-environments
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("optuna-keras")
myenv.docker.enabled = True
#myenv.docker.base_image = None
myenv.python.conda_dependencies = CondaDependencies()
myenv.python.conda_dependencies.add_pip_package("azureml-sdk[notebook]")
myenv.python.conda_dependencies.add_pip_package("optuna")
myenv.python.conda_dependencies.add_pip_package("mysql-connector-python-rf")
myenv.python.conda_dependencies.add_pip_package("tensorflow==1.14")
myenv.python.conda_dependencies.add_pip_package("tensorflow-gpu")
myenv.python.conda_dependencies.add_pip_package("keras")

In [14]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DEFAULT_CPU_IMAGE

src = ScriptRunConfig(source_directory=".", script='keras_mnist.py')
src.run_config.target = cpu_cluster.name
src.run_config.environment = myenv
src.run_config.node_count = 1 
src.run_config.max_run_duration_seconds = 2400

## 実験メトリックの取得開始

In [15]:
optuna_run = experiment.start_logging()

## 最大 Study Job 数

In [16]:
# 最大5並列で Study を実行
num_of_child = 5

## Study 実行

In [17]:
for c in range(num_of_child):
    optuna_run.submit_child(src)

In [20]:
from azureml.widgets import RunDetails
RunDetails(optuna_run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Study 結果集計

In [21]:
study = optuna.load_study(study_name='keras-mnist', storage="mysql+mysqlconnector://{0}:{1}@{2}/{3}".format(user, password, host, dbname))
df = study.trials_dataframe()
df

,number,state,value,datetime_start,datetime_complete,params,system_attrs
,,,,,,c,_number
0,0,TrialState.COMPLETE,0.3413,2019-10-19 15:21:19,2019-10-19 15:22:49,0.000018,0
1,1,TrialState.COMPLETE,0.3659,2019-10-19 15:21:22,2019-10-19 15:22:59,0.000019,1
2,2,TrialState.COMPLETE,0.8553,2019-10-19 15:21:23,2019-10-19 15:23:01,0.000217,2
3,3,TrialState.COMPLETE,0.8125,2019-10-19 15:21:24,2019-10-19 15:22:59,0.000183,3
4,4,TrialState.COMPLETE,0.3700,2019-10-19 15:21:28,2019-10-19 15:23:03,0.000015,4
5,5,TrialState.COMPLETE,0.5707,2019-10-19 15:23:15,2019-10-19 15:24:56,0.000031,5
6,6,TrialState.COMPLETE,0.8967,2019-10-19 15:23:16,2019-10-19 15:25:02,0.000576,6
7,7,TrialState.COMPLETE,0.9069,2019-10-19 15:23:17,2019-10-19 15:25:05,0.000753,7
8,8,TrialState.COMPLETE,0.7117,2019-10-19 15:23:18,2019-10-19 15:25:04,0.000068,8


In [22]:
optuna_run.get_metrics(name="Final test accuracy", recursive=True)

{'optuna-keras-mnist_1571498077_8f431746': {'Final test accuracy': [0.8125,
   0.7117000222206116,
   0.9138000011444092,
   0.8828999996185303,
   0.7821999788284302]},
 'optuna-keras-mnist_1571498091_f17e3718': {'Final test accuracy': [0.34130001068115234,
   0.5706999897956848,
   0.8971999883651733,
   0.8776999711990356,
   0.6916000247001648]},
 'optuna-keras-mnist_1571498104_a75ee4a4': {'Final test accuracy': [0.3659000098705292,
   0.8615000247955322,
   0.913100004196167,
   0.8862000107765198,
   0.7426000237464905]},
 'optuna-keras-mnist_1571498116_901b591b': {'Final test accuracy': [0.8553000092506409,
   0.9068999886512756,
   0.9121999740600586,
   0.8815000057220459,
   0.7656000256538391]},
 'optuna-keras-mnist_1571498127_93d08f03': {'Final test accuracy': [0.3700000047683716,
   0.8967000246047974,
   0.9106000065803528,
   0.8819000124931335,
   0.6747000217437744]}}

## 実行メトリックの取得終了

In [23]:
optuna_run.complete()